# EDA y primer modelo - Grupo03
### Authors
- César López Mantecón - 100472092
- Manuel Gómez-Plana Rodríguez - 100472310

### Repositorio
Esta práctica se ha llevado a cabo en [este repositorio de github](https://github.com/CLopMan/aprendizajeAutomatico-G03)


## Índice
...

## Introducción
En este *notebook* se recoge el Análisis de Datos Exploratorio (EDA, por sus siglas en inglés[1]) además de la selección del primer modelo. Comenzaremos por el EDA. Este proceso constará de los siguientes pasos: ajuste de datos, determinación de características e instancias, análisis de variables e instancias (estudio de la relación entre variables y existencia de valores nulos -- TODO: borrar este paréntesis) y modelización del problema.

Tras el EDA, haremos un análisis de la evaluación *outer* e *inner* junto con la decisión de las métricas usadas. Luego, decidiremos el método de escalado más adecuado usando el KNN como algoritmo, para luego probar varios modelos para poder decidir cual es el mejor.

Ahora, comencemos con el EDA.

---

[1]: IBM, "Análisis de datos exploratorio". IBM. https://www.ibm.com/es-es/topics/exploratory-data-analysis (acceso: 28 de febrero de 2024)

## EDA - 1.Ajuste de datos
En este apartado haremos un pequeño preprocesado de los datos para eliminar la instancias que no son relevantes para el problema propuesto.
Para ello, comenzamos importando el módulo `pandas`. Esto nos permitirá usar la herramienta de *dataframes* con el fin anterior.

In [ ]:
import pandas as pd

Gracias a `pandas` podemos echar un primer vistazo a los datos, primero importando el csv y luego comprobando la cantidad de instancias iniciales.

In [ ]:
csv = pd.read_csv("wind_ava.csv")
print("nº filas = " + str(len(csv)) + "\nnº columnas = " + str(len(csv.columns)))

Comprobamos que este *feature space* es un espacio 552-dimensional con 4748 instancias.

A continuación, generamos un fichero cuyas columnas sean las de la localización *Sotavento*, es decir, la número 13, reduciendo las dimensiones de este *dataframe*.

In [ ]:
csv = pd.read_csv("wind_ava.csv")
csv = csv.filter(regex='13$|energy|datetime')

Como podemos observar, el *feature space* disminuye a un espacio 24-dimensional, con las mismas 4748 instancias previas. Con esto, concluimos el ajuste del dataframe para reducirlo a los datos relevantes a la localización Sotavento.

# EDA - 2.Determinación de características e instancias
En este apartado, analizaremos los tipos de datos que conforman el *feature space*, describiendo qué valores son numéricos y cuáles categóricos. En el siguiete código, mostramos una instancia del *dataframe* para poder observar cada *feature*.

In [ ]:
first_row = csv[0:1]
first_row.to_csv("sample.csv")

Observamos que las 24 instancias son **numéricas** en coma flotante, destacando una marca de tiempo en la columna *daytime*. Para poder visualizar mejor la información, nos hemos valido de la herramienta *LibreOffice Calc*.

## EDA - 3. Análisis de variables e Instancias

A continuación, estudiaremos el conjunto de datos para comprobar la existencia de relaciones entre variables, con el fin de limpiar el conjunto de datos que no sean relevantes para el estudio. Para ello, primero haremos un breve análisis de los datos para comprobar la existencia de valores nulos o atípicos.

### Existencia de valores nulos
Comprobamos mediante el siguiente código que **no existen valores nulos en el set de datos** .

In [ ]:
for i in range(len(csv)):
 for j in range(len(csv.columns)):
  if (pd.isnull(csv.iloc[i, j])):
    print("Dataframe column", str(j), "in row", i," is null.")

### Existencia de valores atípicos



### Estudio de correlaciones


### Existencia de columnas constantes

Para comprobar la existencia de columnas constantes. Debido a la naturaleza de los datos, se considerará que dos valores son distintos si difieren en más de un 1% del valor más grande. A través del siguiente código vemos que **no existen columnas constantes**.

In [ ]:
for j in range(0, 1, 1):
 same_date = True
 same_day = True
 i = 0
 while((same_date or same_day) and i < len(csv)-1):
  if (csv.iloc[i, j] != csv.iloc[i+1,j]):
   same_date = False
  if (csv.iloc[i, j][0:10] != csv.iloc[i+1,j][0:10]):
   same_day = False
  i += 1
 if same_date:
  print("The datetime values", str(j), "are constant.")
 if same_day:
  print("The values", str(j), "are from the same day")

for j in range(1, len(csv.columns), 1):
 same = True
 i = 0
 while (same and i < len(csv)-1):
  if (csv.iloc[i+1, j] < 0.99*csv.iloc[i, j] or csv.iloc[i+1,j] > 1.01*csv.iloc[i,j]):
   same = False
  i += 1
 if same:
  print("The values of the column", str(j), "are constant.")

# EDA - 4.Tipo de problema
Tras estos análisis, debemos instanciar este problema como un problema de **Clasificación** o de **Regresión**. Lo primero que debemos saber para realizar esta clasificación, es saber la variable que tenemos que predecir, es decir, **energía**. Con los bloques de código anteriores, hemos visto que la energía es un valor discreto, por lo que podemos asumir que el modelo intenta resolver un problema de **Regresión**.

Por esto, podemos decir, adicionalmente, que al estimar un valor según unas variables, se trata de un problema del tipo aprendizaje **atributo-valor*. Además, como tenemos los valores de **energía**, podemos decir que se trata de un problema de aprendizaje supervisado.